In [ ]:
!pip install datasets

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd
from datasets import Dataset
from datasets import ClassLabel
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import random
import argparse
import pickle
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import os
import torch
import time
import os
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
#Model fine-tuning
!python model_training.py --train_data /content/drive/MyDrive/multilingual-hate-speech/data/multi_duplicate_all.csv --test_data_en /content/drive/MyDrive/multilingual-hate-speech/data/EN_test.csv --test_data_it /content/drive/MyDrive/multilingual-hate-speech/data/IT_test.csv --test_data_slo /content/drive/MyDrive/multilingual-hate-speech/data/SLO_test.csv --output_dir model_duplicate_all_NORMAL_epoch3 --results_dir results_duplicate_all_NORMAL_epoch3 --batch_size 12 --epochs 1

2024-05-22 16:29:52.666548: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 16:29:52.719569: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 16:29:52.719613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 16:29:52.721337: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 16:29:52.729677: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/multilingual-hate-speech/model_duplicate_all_noreweighting_3epoch')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/multilingual-hate-speech/model_duplicate_all_noreweighting_3epoch')

In [ ]:
#Model evaluation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

df_test_en = pd.read_csv('/content/drive/MyDrive/multilingual-hate-speech/data/EN_test.csv')

# English test set
df_hug_en = pd.DataFrame()
df_hug_en['label'] = df_test_en['Type']
df_hug_en['text'] = df_test_en['Text']


text_list = df_hug_en['text'].tolist()  # Ensure it is a list of strings
tokenized_test_set = tokenizer(text_list, padding="max_length",
                               truncation=True, return_tensors="pt")


# Convert tokenized data into a TensorDataset
test_dataset = TensorDataset(tokenized_test_set['input_ids'], tokenized_test_set['attention_mask'])

# Create a DataLoader for handling batches
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Prediction in batches
all_logits = []
all_predictions = []
model.eval()  # Ensure the model is in evaluation mode

with torch.no_grad():
    for batch in test_loader:
        batch_input_ids, batch_attention_mask = batch[0].to(device), batch[1].to(device)
        outputs = model(batch_input_ids, batch_attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predictions = torch.argmax(probabilities, dim=-1)

        # Move logits and predictions back to CPU for storage/analysis
        all_logits.append(logits.cpu())
        all_predictions.append(predictions.cpu())

# Concatenate all collected batches
logits = torch.cat(all_logits, dim=0)
predictions = torch.cat(all_predictions, dim=0)

predictions = predictions.cpu().numpy()

#print("Predictions:", predictions)

cm = confusion_matrix(df_hug_en['label'], predictions)
print(cm)
print('Accuracy:' + str(accuracy_score(df_hug_en['label'], predictions)))
print('Micro F1 score:' + str(f1_score(df_hug_en['label'], predictions, average='micro')))
print('Macro F1 score:' + str(f1_score(df_hug_en['label'], predictions, average='macro')))
print('Weighted F1 score:' + str(f1_score(df_hug_en['label'], predictions, average='weighted')))
f1_scores = f1_score(df_hug_en['label'], predictions, average=None)
print("F1 '0 appropriate: '" + str(f1_scores[0]))
print("F1 '1 inappropriate: '" + str(f1_scores[1]))
print("F1 '2 offensive: '" + str(f1_scores[2]))
print("F1 '3 violent: '" + str(f1_scores[3]))
print('Recall violent class:' + str(recall_score(df_hug_en['label'], predictions, labels=[3], average=None)[0]))

metrics = {
        'confusion_matrix': cm,
        'accuracy': accuracy_score(df_hug_en['label'], predictions),
        'micro_f1_score': f1_score(df_hug_en['label'], predictions, average='micro'),
        'macro_f1_score': f1_score(df_hug_en['label'], predictions, average='macro'),
        'weighted_f1_score': f1_score(df_hug_en['label'], predictions, average='weighted'),
        'f1_scores': {
            '0_appropriate': f1_scores[0],
            '1_inappropriate': f1_scores[1],
            '2_offensive': f1_scores[2],
            '3_violent': f1_scores[3]
        },
        'recall_3_violent': recall_score(df_hug_en['label'], predictions, labels=[3], average=None)[0]
    }

results_dir = "/content/drive/MyDrive/multilingual-hate-speech/model_duplicate_all_noreweighting_3epoch/results_duplicate_all_NORMAL_epoch3"

# Save metrics to a pickle file in the specified directory
metrics_file_path = os.path.join(results_dir, 'metrics_xlmr_en.pkl')
with open(metrics_file_path, 'wb') as file:
    pickle.dump(metrics, file)

# Save predictions to a pickle file in the specified directory
metrics_file_path = os.path.join(results_dir, 'predictions_xlmr_en.pkl')
with open(metrics_file_path, 'wb') as file:
    pickle.dump(predictions, file)

tensor([0, 0, 0,  ..., 0, 0, 0])
Predictions: [0 0 0 ... 0 0 0]
[[13650    45  2203    36]
 [   45    59    77     2]
 [ 1090    34  4144    42]
 [   19     0    33    24]]
Accuracy:0.8313723666465145
Micro F1 score:0.8313723666465145
Macro F1 score:0.5566905099252667
Weighted F1 score:0.8361346214839891
F1 '0 appropriate: '0.8881514737458519
F1 '1 inappropriate: '0.36760124610591904
F1 '2 offensive: '0.7043426531826295
F1 '3 violent: '0.26666666666666666
Recall violent class:0.3157894736842105


In [ ]:
#Qualitative analysis pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Input sentence
input_sentence = "The JEWZ are Market Manipulators Hate them freaking Jews !! Hitler should have killed them all !!!"  # Replace with your input sentence

# Tokenizing the input sentence
tokenized_input = tokenizer(input_sentence, padding="max_length", truncation=True, return_tensors="pt")

input_ids = tokenized_input['input_ids'].to(device)
attention_mask = tokenized_input['attention_mask'].to(device)

# Model prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    prediction = torch.argmax(probabilities, dim=-1)

predicted_label = prediction.item()
print(f"Predicted label: {predicted_label}")

Predicted label: 3
